In [ ]:
import concurrent.futures
import glob
import time
import sys
import os
import pandas as pd
from PyPDF2 import PdfFileReader
import io
import re
import csv
import numpy as np
from datetime import datetime
import math
from pdf2image import convert_from_bytes, convert_from_path
from PIL import Image
import pytesseract
import cv2
from PIL import Image

In [ ]:


def binarization(image):
    image = image.convert('RGB')

    npimage = np.asarray(image).astype(np.uint8)  

    npimage[:, :, 0] = 0
    npimage[:, :, 2] = 0

    im = cv2.cvtColor(npimage, cv2.COLOR_RGB2GRAY) 

    ret, thresh = cv2.threshold(im, 80, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU) 

    binimage = Image.fromarray(thresh)
    return binimage


def get_pages_as_images(file):
    #images = convert_from_path('{}'.format(file), 250, grayscale=True)
    images = convert_from_bytes(file, 300, grayscale=True)
    num_page = 1
    for image in images:
        image = binarization(image)
    return images



def do_ocr(data):

    img = data[1]
    text = pytesseract.image_to_string(img,lang='por')
    
    '''
    out_dir = ""
    out_file = 'tmp/page-{}.txt'.format(data[0])
    out_path = out_dir + out_file
    f = open(out_path,"w")
    f.write("%s" %text)
    f.close()
    '''
    return text

def ocr_job(image_list):
    os.environ['OMP_THREAD_LIMIT'] = '1'
    num_workers = os.cpu_count()
    results = {}
    with concurrent.futures.ProcessPoolExecutor(max_workers=num_workers - 1) as executor:      
        data = []
        for i in range(len(image_list)):
            data.append((i, image_list[i]))
        for (page, file), text in zip(data, executor.map(do_ocr, data)):
            #print('page {}'.format(page))
            results[int(page)] = text
                
    return results

In [ ]:
print(os.cpu_count())

In [ ]:
start = time.time()
text = ocr_job(pdf)
end = time.time()